In [5]:
import pandas as pd
from dbfread import DBF

df_list = []

columns = ['']

for i in range(13, 23):
    file = DBF(f'data/DOPB20{i}.dbf', load=True)
    df = pd.DataFrame(file,)
    df_list.append(df)

In [8]:
df_concat = pd.DataFrame()

for df in df_list:
    df_concat = pd.concat([df_concat, df])

In [46]:
from numpy import nan

df_concat = (df_concat
 .drop_duplicates()
 .replace('', nan)
 #.filter(['ESC2010', 'CIRCOBITO'])
 .query('CIRCOBITO == "3"')
 .astype({'ESC2010': float})
)

In [58]:
qtd_linhas = df_concat.shape[0]

(df_concat
 .groupby('ESC2010')['CONTADOR'].count()
 .apply(lambda x: (x/qtd_linhas) * 100).sum()
)

np.float64(61.76258126655429)